# Neural Network

Welcome to your next lab! You will solve problem of handwritten digits recognition using Neural Network.


**You will learn to:**
- Build the general architecture of a learning algorithm with OOP in mind:
    - Helper utilities
        - Sigmoid (and it's derivative)
        - One-Hot
        - Cost Function
        - Regularization
    - Neural Network Class
        - Forward propagation
        - Backward propagation
        - Upgrade parameters
    - Main Model Classes
        - Training
        - Prediction 

## 1 - Packages ##

First, let's run the cell below to import all the packages that you will need during this assignment. 
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.
- [seaborn](https://seaborn.pydata.org/) is a Python visualization library which provides a high-level interface for drawing attractive statistical graphics.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

## 2 - Overview of the Problem set ##

**Problem Statement**: 
We'll use the MNIST data set, which contains tens of thousands of scanned images of handwritten digits, together with their correct classifications. MNIST's name comes from the fact that it is a modified subset of two data sets collected by NIST, the United States' National Institute of Standards and Technology.

The MNIST dataset contains 60,000 images. These images are scanned handwriting samples from 250 people, half of whom were US Census Bureau employees, and half of whom were high school students. The images are greyscale and 28 by 28 pixels in size. 
So, you are given a dataset  containing:
    - a training set of m_train examples labeld as 0-9
    - a test set of m_test examples labeld as 0-9
    - each example is an array of length 784 (28 * 28) which represents image of handwritten digit.
    
You will build an algorithm that can recognize handwritten digits.

Let's get more familiar with the dataset. Load the data by running the following code.

In [0]:
# Read the MNIST dataset from ubyte files 

def read_mnist(images_path, labels_path):
    import struct
    import os
    with open(labels_path, 'rb') as p:
        magic, n = struct.unpack('>II', p.read(8))
        labels = np.fromfile(p, dtype=np.uint8)
    with open(images_path, 'rb') as p:
        magic, num, rows, cols = struct.unpack(">IIII", p.read(16))
        images = np.fromfile(p, dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

# Shuffle dataset

def shuffle_data(features, labels, random_seed=42):
    assert len(features) == len(labels)

    if random_seed:
        np.random.seed(random_seed)
    idx = np.random.permutation(len(features))
    return [a[idx] for a in [features, labels]] 

# Loading data

def load_data():     
    X, y = read_mnist('samples/train-images-idx3-ubyte', 'samples/train-labels-idx1-ubyte')
    X, y = shuffle_data(X, y, random_seed=42)
    train_set_x, train_set_y = X[:5000], y[:5000]
    test_set_x, test_set_y = X[5000:], y[5000:]
    
    test_set_x = test_set_x.reshape(test_set_x.shape[0], -1).T
    train_set_x = train_set_x.reshape(train_set_x.shape[0], -1).T
    train_set_y = train_set_y.reshape((1, train_set_y.shape[0]))
    test_set_y = test_set_y.reshape((1, test_set_y.shape[0]))
    
    return train_set_x, test_set_x, train_set_y, test_set_y

Let's create train and test datasets:

In [0]:
train_set_x, test_set_x, train_set_y, test_set_y = load_data()
print('train set shapes: ', train_set_x.shape, train_set_y.shape)
print('test set shapes: ', test_set_x.shape, test_set_y.shape)

**Expected Output**: 

<table>
    <tr>
        <td> **train set shapes: ** </td>
       <td> (784, 5000) (1, 5000)</td>
        
    </tr>
    
    <tr>
        <td> **test set shapes: ** </td>
       <td> (784, 55000) (1, 55000)</td>
        
    </tr>
    

</table>

### Data exploration ###

Let's build a function to check how the data looks like:

In [0]:
def plot_digit(x_set, y_set, idx):
    img = x_set.T[idx].reshape(28,28)
    plt.imshow(img, cmap='Greys',  interpolation='nearest')
    plt.title('true label: %d' % y_set.T[idx])
    plt.show()

In [0]:
plot_digit(train_set_x, train_set_y, idx=1)

In [0]:
plot_digit(train_set_x, train_set_y, idx=3)

## 3 - Helper Functions

For begining we need to implement some special functions.

### Sigmoid (and it's derivative)

Any layer of a neural network can be considered as an Affine Transformation followed by application of a non linear function. A vector is received as input and is multiplied with a matrix to produce an output , to which a bias vector may be added before passing the result through an activation function such as sigmoid.

$$Input = x \quad Output = f(Wx+b)\tag{1}$$

The sigmoid function is used quite commonly in the realm of deep learning, at least it was until recently. It has distinct **S** shape and it is a differentiable real function for any real input value. Additionally, it has a positive derivative at each point. More importantly, we will use it as an activation function for the hidden layer of our model. Here's how it is defined:

$$\sigma (x) = \frac{1}{1+e^{-x}}\tag{2}$$

Here is first derivative (which we will use during the backpropagation step of our training algorithm). It has the following formula:

$$\frac{d\sigma (x)}{d(x)} = \sigma (x)\cdot (1-\sigma(x))\tag{3}$$

In [0]:
# GRADED CLASS: Sigmoid

class Sigmoid:
    def __call__(self, z):
        """
        Compute the sigmoid of z

        Arguments:
        z -- scalar or numpy array of any size.

        Return:
        sigmoid(z)
        """
        ### START CODE HERE ### (≈ 1 line of code)
        return 1 / (1 + np.exp(-z))
        ### END CODE HERE ###
    
    def prime(self, z):
        """
        Compute the derivative of sigmoid of z

        Arguments:
        z -- scalar or numpy array of any size.

        Return:
        Sigmoid prime
        """
        ### START CODE HERE ### (≈ 1-2 lines of code)
        return np.exp(-z)/((1+np.exp(-z))**2)
        ### END CODE HERE ###

In [0]:
x = np.linspace(-10., 10., num=100)
sig = Sigmoid()(x)
sig_prime = Sigmoid().prime(x)
print('sigmoid:')
print(sig[:5])
print('sigmoid_prime: ')
print(sig_prime[:5])

**Expected Output**: 

<table>
    <tr>
       <td> **sigmoid: ** </td>
       <td> [  4.53978687e-05   5.55606489e-05   6.79983174e-05   8.32200197e-05
   1.01848815e-04] </td>
    </tr>
    <tr>
       <td> **sigmoid_prime: ** </td>
       <td> [  4.53958077e-05   5.55575620e-05   6.79936937e-05   8.32130942e-05
   1.01838442e-04] </td>
    </tr>

</table>

Now let's look at the picture:

In [0]:
plt.plot(x, sig, label="sigmoid")
plt.plot(x, sig_prime, label="sigmoid prime")
plt.xlabel("x")
plt.ylabel("y")
plt.legend(prop={'size' : 16})
plt.show()

The derivative shows the rate of change of a function. We can use it to determine the "slope" of that function. The highest rate of change for the sigmoid function is when $x=0$, as it is evident from the derivative graph.

### One-Hot ###

For convenience let's represent input labels $y$ as a 10-dimensional vector. For example, if a particular training image, $x$, depicts a 6, then $y(x)=(0,0,0,0,0,0,1,0,0,0)^T$ is the desired output from the network.

So, let's encode our labels as one-hot vectors:

In [0]:
# GRADED FUNCTION: one_hot

def one_hot(Y, n_classes):
    """
    Encode labels into a one-hot representation

    Arguments:
    Y -- array of input labels of shape (1, n_samples)
    n_classes -- number of classes

    Returns:
    onehot, a matrix of labels by samples. For each column, the ith index will be 
        "hot", or 1, to represent that index being the label; shape - (n_classes, n_samples)
    """
    ### START CODE HERE ### (≈ 1-4 lines of code)
    res = np.zeros((n_classes,len(Y[0])))
    for i, el in enumerate(Y[0]):
        res[el][i] = 1
    return res 
    ### END CODE HERE ###

In [0]:
print('encoded:')
print(one_hot(np.asarray([1, 2, 3, 4, 3]).reshape(1, 5), 5))

**Expected Output**: 

<table>
    <tr>
        <td> **encoded: ** </td>
       <td> [[ 0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.]
 [ 0.  0.  0.  1.  0.]]</td>
    </tr>
    

### Cost Function ###

Here is cost function for our model:
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \sum\limits_{j = 0}^{n}\large{(} \small y^{(i)}_{j}\log\left(a^{[2] (i)}_{j}\right) + (1-y^{(i)}_{j})\log\left(1- a^{[2] (i)}_{j}\right)  \small\large{)}\tag{4}$$
where 

$m$ - number of examples;

$n$ - number of classes;

$y$ - input labels;

$a$ - output of hidden layer.


We will use cross entropy loss:
$- \sum\limits_{i=0}^{m}  y^{(i)}\log(a^{[2](i)})\tag{5}$

In [0]:
# GRADED FUNCTION: compute_cost

def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost given in equation (4)
    
    Arguments:
    A2 -- sigmoid output of the hidden layer activation, of shape (classes, n_examples)
    Y -- labels of shape (classes, n_examples)
    
    Returns:
    cost -- cross-entropy cost given equation (4)
    """
    
    m = Y.shape[1] # number of examples

    # Compute the cross-entropy cost
    ### START CODE HERE ###
    cost = -1/m * sum( sum(Y * np.log(A2) + (1-Y) * np.log(1 - A2) ))
    ### END CODE HERE ###
    
    return cost

In [0]:
np.random.seed(1)
labels = (np.random.randn(2, 3) > 0)
a_h = (np.array([[ 0.5 ,  0.49,  0.5], [0.1 ,  0.2,  0.3]]))
print("cost:" + str(compute_cost(a_h, labels)))

**Expected Output**: 

<table style="width:25%">
    <tr>
       <td> **cost: ** </td>
       <td> 1.3770374288... </td>
    </tr>
    

</table>

### Regularization

In order to create less complex (parsimonious) model when you have a large number of features in your dataset, some of the Regularization techniques used to address over-fitting and feature selection are:

1. L1 Regularization

2. L2 Regularization

A regression model that uses L1 regularization technique is called Lasso Regression and model which uses L2 is called Ridge Regression.

Lasso Regression (Least Absolute Shrinkage and Selection Operator) adds “absolute value of magnitude” of coefficient as penalty term to the cost function.

Ridge regression adds “squared magnitude” of coefficient as penalty term to the loss function. 
The cost is then computed by summing squared diff over all training examples.
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \sum\limits_{j = 0}^{n}\large{(} \small y^{(i)}_{j}\log\left(a^{[2] (i)}_{j}\right) + (1-y^{(i)}_{j})\log\left(1- a^{[2] (i)}_{j}\right)  \small\large{)} + \frac{\lambda_1}{m}( ||w_1||_1 + ||w_2||_1) + \frac{\lambda_2}{2m} (||w_1||_2^2+||w_2||_2^2) \tag{6}$$

To calculate norms use "entrywise" norm.

In [0]:
# GRADED CLASS: Regularisation

class Regularization:
    """ 
    Regularization class

    Arguments:
    lambda_1 -- regularization coeficient for l1 regularization
    lambda_2 -- regularization coeficient for l2 regularization
    """
    def __init__(self, lambda_1, lambda_2):
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        
        
    def l1(self, W1, W2, m):
        """ 
        Compute l1 regularization part

        Arguments:
        W1 -- weigts of shape (n_hidden_units, n_features) 
        W2 -- weigts of shape (output_size, n_hidden_units) 
        m -- n_examples

        Returns:
        l1_term -- float, check formula (6)
        """
        ### START CODE HERE ###
        return self.lambda_1/m * (np.linalg.norm(W1,1) + np.linalg.norm(W2,1))
        ### END CODE HERE ###
        
    def l1_grad(self, W1, W2, m):
        """ 
        Compute l1 regularization term

        Arguments:
        W1 -- weigts of shape (n_hidden_units, n_features) 
        W2 -- weigts of shape (output_size, n_hidden_units) 
        m -- n_examples

        Returns:
         dict with l1_grads "dW1" and "dW2"
            which are grads by corresponding weights
        """
        ### START CODE HERE ###
        return {'dW1':self.lambda_1/m * np.sign(W1),'dW2':self.lambda_1/m * np.sign(W2)}
        ### END CODE HERE ###

    def l2(self, W1, W2, m):
        """ 
        Compute l2 regularization term

        Arguments:
        W1 -- weigts of shape (n_hidden_units, n_features) 
        W2 -- weigts of shape (output_size, n_hidden_units) 
        m -- n_examples

        Returns:
        l2_term: float, check formula (6)
        """
        ### START CODE HERE ###
        return self.lambda_2/(2*m) * (np.linalg.norm(W1,2)**2 + np.linalg.norm(W2,2)**2)
        ### END CODE HERE ###
        
    def l2_grad(self, W1, W2, m):
        """ 
        Compute l2 regularization term

        Arguments:
        W1 -- weigts of shape (n_hidden_units, n_features) 
        W2 -- weigts of shape (output_size, n_hidden_units) 
        m -- n_examples

        Returns:
        l2_grads: dict with keys "dW1" and "dW2"
        """
        ### START CODE HERE ###
        return {'dW1':self.lambda_2/m * W1,'dW2':self.lambda_2/m * W2}
        ### END CODE HERE ###

In [0]:
reg = Regularization(0.3, 0.2)
print('l1 reg. term =', reg.l1(np.array([0.1, -0.2, 0.3, 0.4]), np.array([-0.5, 0.6, 0.7, 0.8]), 5))
print('l2 reg. term =', reg.l2(np.array([0.1, -0.2, 0.3, 0.4]), np.array([-0.5, 0.6, 0.7, 0.8]), 5))
print('l1 reg. gradient =', reg.l1_grad(np.array([0.1, -0.2, 0.3, 0.4]), np.array([-0.5, 0.6, 0.7, 0.8]), 5))
print('l2 reg. gradient =', reg.l2_grad(np.array([0.1, -0.2, 0.3, 0.4]), np.array([-0.5, 0.6, 0.7, 0.8]), 5))

**Expected Output**: 

<table>
    <tr>
       <td> **l1 reg. term** </td>
       <td> 0.216 </td>
    </tr>
    <tr>
       <td> **l2 reg. term** </td>
       <td>  0.0408 </td>
    </tr>
    <tr>
       <td> **l1 reg. gradient** </td>
       <td> {'dW1': array([ 0.06, -0.06,  0.06,  0.06]),  </td>
       <td>  'dW2': array([-0.06,  0.06,  0.06,  0.06])} </td>
    </tr>
    <tr>
       <td> **l2 reg. gradient** </td>
       <td> {'dW1': array([ 0.004, -0.008,  0.012,  0.016]),
  </td>
      <td> 'dW2': array([-0.02 ,  0.024,  0.028,  0.032])}
      </td>
    </tr>


</table>

## 3 - Neural Network. General Architecture ##

To recognize digits we will use a two-layer neural network. Here is model of our network:
    <img src="samples/arc.png" style="width:620px;height:500px;">
    
The input layer of the network contains neurons encoding the values of the input pixels. As discussed earlier, training data consist of many 28 by 28 pixel images of scanned handwritten digits, and so the input layer contains $784=28×28$ neurons. The input pixels are greyscale, with a value of 0.0 representing white, a value of 1.0 representing black, and in between values representing gradually darkening shades of grey.

The first layer of the network is a hidden layer. We denote the number of neurons in this hidden layer by n, and we'll experiment with different values for n. The example shown illustrates a small hidden layer, containing just $n=15$ neurons.

The output layer of the network contains 10 neurons. If the first neuron fires, i.e., has an output $≈1$, then that will indicate that the network thinks the digit is a 0. If the second neuron fires then that will indicate that the network thinks the digit is a 1, and so on. A little more precisely, we number the output neurons from 0 through 9, and figure out which neuron has the highest activation value. If that neuron is, say, neuron number 6, then our network will guess that the input digit was a 6. And so on for the other output neurons.

Each training input $x$ is a $28×28=784$-dimensional vector. Each entry in the vector represents the grey value for a single pixel in the image. 


### 3.1 - Neural Network Class ####

In this block we will implement class *NeuralNetwork*, which will perform forward and backward propagation, and update parameters.  

**Main steps to build neural network:**
    1. Initialize the model's parameters
    2. Loop:
        - Implement forward propagation
        - Compute loss
        - Implement backward propagation to get the gradients
        - Update parameters (gradient descent)
        
**1. Initializing the model's parameters**

You need to do following steps:
- Make sure your parameters' sizes are right. Refer to the neural network figure above if needed.        
- Initialize the weights matrices with random values from the “standard normal” distribution, where sigma = 0.01, mu = 0. 
- Initialize the bias vectors as zeros. 
   
**2. Loop**

   - *Forward propagation*
   
In forward propagation we use the data and the weights of the network to compute a prediction, so during this step we just multiply the matrix containing our training data with the matrix of the weights of the hidden layer.

For one example $x^{(i)}$:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{8}$$ 
$$a^{[1] (i)} = \sigma(z^{[1] (i)})\tag{9}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{10}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{11}$$
$$y^{(i)}_{prediction} = argmax(a^{[2](i)}) \tag{12}$$

Definition: 

$W^{[1]}$: weights matrix 1

$W^{[2]}$: weights matrix 2

$b^{[1]}$: bias 1

$b^{[2]}$: bias 2

$z^{[1] (i)}$ : input of the hidden layer

$a^{[1] (i)}$: activation of the hidden layer

$z^{[2] (i)}$: input of the output layer 

$a^{[2] (i)}$: activation of the output layer

$\hat{y}^{(i)}$: array of predictions

   - *Backward propagation*
   
In backpropagation step we apply the activation function (sigmoid) to the result and multiply that with the weight matrix of the output layer. 

<table>
    <tr>
       <td> $dz^{[2]} = a^{[2]} - y$  </td>
       <td> $dZ^{[2]} = A^{[2]} - Y$ </td>
    </tr>
    <tr>
       <td> $dW^{[2]} = dz^{[2]}a^{[1]T} + \frac{\lambda_1}{m}sign(W^{[2]}) + \frac{\lambda_2}{m}W^{[2]}$ </td>
       <td>  $dW^{[2]} = \frac{1}{m}dZ^{[2]}A^{[1]T} + \frac{\lambda_1}{m}sign(W^{[2]}) + \frac{\lambda_2}{m}W^{[2]}$ </td>
       
    </tr>
    <tr>
       <td> $db^{[2]} = dz^{[2]}$ </td>
       <td> $db^{[2]} = \frac{1}{m}np.sum(dZ^{[2]}, axis=1, keepdims=True)$ </td>
    </tr>
    <tr>
       <td> $dz^{[1]} = W^{[2]T}dz^{[2]}*\sigma'(z^{[1]})$ </td>
       <td> $dz^{[1]} = W^{[2]T}dZ^{[2]}*\sigma'(Z^{[1]})$ </td>
    </tr>
    <tr>
       <td> $dW^{[1]} = dz^{[1]}x^T + \frac{\lambda_1}{m}sign(W^{[1]}) + \frac{\lambda_2}{m}W^{[1]}$ </td>
       <td> $dW^{[1]} = \frac{1}{m}dZ^{[1]}X^T + \frac{\lambda_1}{m}sign(W^{[1]}) + \frac{\lambda_2}{m}W^{[1]}$  </td>
    </tr>
    <tr>
       <td> $db^{[1]} = dZ^{[1]}$ </td>
       <td> $db^{[1]} = \frac{1}{m}np.sum(dZ^{[1]}, axis=1, keepdims=True)$  </td>
    </tr>
</table>

   - *Update Parameters*
   
Next, the error is computed based on the prediction and the provided labels. The final step propagates the error through the network, starting from the final layer. Thus, the weights get updated based on the error, little by little. To implement this method we will use gradient descent. You have to use (dW1, db1, dW2, db2) in order to update (W1, b1, W2, b2).

**General gradient descent rule**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ where $\alpha$ is the learning rate and $\theta$ represents a parameter.

**Illustration**: The gradient descent algorithm with a good learning rate (converging) and a bad learning rate (diverging). Images courtesy of Adam Harley.

<img src="samples/sgd.gif" style="width:400;height:400;"> <img src="samples/sgd_bad.gif" style="width:400;height:400;">


Implement class `NeuralNetwork` in the cell below: 

In [0]:
# GRADED CLASS NeuralNetwork

class NeuralNetwork:
    """
    Arguments:
    n_features: int -- Number of features
    n_hidden_units: int -- Number of hidden units
    n_classes: int -- Number of classes
    learning_rate: float
    reg: instance of Regularization class
    """
    def __init__(self, n_features, n_hidden_units, n_classes , learning_rate, reg=Regularization(0.1, 0.2), sigm=Sigmoid()):
        self.n_features = n_features
        self.n_classes = n_classes
        self.learning_rate = learning_rate
        self.n_hidden_units = n_hidden_units
        self.reg = reg
        self.sigm = sigm
        self.W1 = None
        self.b1 = None
        self.W2 = None
        self.b2 = None
        
        self.initialize_parameters()

    def initialize_parameters(self):
        """
        W1 -- weight matrix of shape (self.n_hidden_units, self.n_features)
        b1 -- bias vector of shape (self.n_hidden_units, 1)
        W2 -- weight matrix of shape (self.n_classes, self.n_hidden_units)
        b2 -- bias vector of shape (self.n_classes, 1)
        """
        np.random.seed(42) 
    
        ### START CODE HERE ### 
        self.W1 = np.random.normal(0, 0.01, (self.n_hidden_units, self.n_features))
        self.b1 = np.zeros((self.n_hidden_units, 1))
        self.W2 = np.random.normal(0, 0.01, (self.n_classes, self.n_hidden_units))
        self.b2 = np.zeros((self.n_classes, 1))
        ### END CODE HERE ###

    def forward_propagation(self, X):
        """
        Arguments:
        X -- input data of shape (number of features, number of examples)
        
        Returns:
        dictionary containing "Z1", "A1", "Z2" and "A2"
        """
        # Implement Forward Propagation to calculate A2 (probabilities)
        ### START CODE HERE ### 
        Z1 = np.dot(self.W1, X) + self.b1
        A1 = self.sigm(Z1)
        Z2 = np.dot(self.W2, A1) + self.b2
        A2 = self.sigm(Z2)
        ### END CODE HERE ###

        return {
            'Z1': Z1,
            'A1': A1,
            'Z2': Z2,
            'A2': A2
        }
    
    def backward_propagation(self, X, Y, cache):
        """
        Arguments:
        X -- input data of shape (number of features, number of examples)
        Y -- one-hot encoded vector of labels with shape (n_classes, n_samples)
        cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"

        Returns:
        dictionary containing gradients "dW1", "db1", "dW2", "db2"
        """
        m = X.shape[1]
        
        # Retrieve also A1 and A2 from dictionary "cache".
        ### START CODE HERE ### 
        A1 = cache['A1']
        A2 = cache['A2']
        Z1 = cache['Z1']
        Z2 = cache['Z2']
        ### END CODE HERE ###

        # Backward propagation: calculate dW1, db1, dW2, db2. 
        ### START CODE HERE ###
        W1 = self.W1
        W2 = self.W2
        dZ2 = A2 - Y
        dW2 = 1/m *np.dot(dZ2, np.transpose(A1)) + self.reg.lambda_1/m * np.sign(W2) + self.reg.lambda_2/m * W2
        db2 = 1/m * np.sum(dZ2,axis = 1,keepdims = True)
        dZ1 = np.dot(np.transpose(W2), dZ2) * self.sigm.prime(Z1)
        dW1 = 1/m*np.dot(dZ1, np.transpose(X)) + self.reg.lambda_1/m * np.sign(W1) + self.reg.lambda_2/m * W1
        db1 = 1/m * np.sum(dZ1,axis = 1,keepdims = True)
        ### END CODE HERE ###

        return {
            'dW1': dW1,
            'db1': db1,
            'dW2': dW2,
            'db2': db2
        }

    def update_parameters(self, grads):
        """
        Updates parameters using the gradient descent update rule 

        Arguments:
        grads -- python dictionary containing gradients "dW1", "db1", "dW2", "db2"
        """
        # Retrieve each gradient from the dictionary "grads"

        ### START CODE HERE ### 
        dW1 = grads['dW1']
        db1 = grads['db1']
        dW2 = grads['dW2']
        db2 = grads['db2']
        ## END CODE HERE ###

        # Update each parameter
        ### START CODE HERE ### 
        self.W1 = self.W1 - self.learning_rate * dW1 
        self.W2 = self.W2 - self.learning_rate * dW2 
        self.b1 = self.b1 - self.learning_rate * db1 
        self.b2 = self.b2 - self.learning_rate * db2 
        ### END CODE HERE ###

Init Neural Network class:

In [0]:
nn_test = NeuralNetwork(2, 4, 3, 0.01)
x_test = np.asarray([[1, 2, 1, 1], [2, 1, 2, 1]])
y_test = np.asarray([1, 2, 1, 0]).reshape(1, 4)

Initialize parameters test:

In [0]:
print('W1:', nn_test.W1)
print('b1:', nn_test.b1)
print('W2:', nn_test.W2)
print('b2:', nn_test.b2)

**Expected Output**: 

<table>

    <tr>
       <td> **W1** </td>
       <td> [[ 0.00496714 -0.00138264]
 [ 0.00647689  0.0152303 ]
 [-0.00234153 -0.00234137]
 [ 0.01579213  0.00767435]] </td>
    </tr>
    <tr>
       <td> **b1** </td>
       <td> [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]] </td>
    </tr>
    <tr>
       <td> **W2** </td>
       <td> [[-0.00469474  0.0054256  -0.00463418 -0.0046573 ]
 [ 0.00241962 -0.0191328  -0.01724918 -0.00562288]
 [-0.01012831  0.00314247 -0.00908024 -0.01412304]] </td>
    </tr>
    <tr>
       <td> **b2** </td>
       <td> [[0.]
 [0.]
 [0.]] </td>
</table>

Forward method test:

In [0]:
cache = nn_test.forward_propagation(x_test)
print('Z1:',cache['Z1'])
print('A1:',cache['A1'])
print('Z2:',cache['Z2'])
print('A2:',cache['A2'])

**Expected Output**: 

<table>
    <tr>
       <td> **Z1** </td>
       <td> [[ 0.00220186  0.00855164  0.00220186  0.0035845 ]
 [ 0.03693748  0.02818407  0.03693748  0.02170718]
 [-0.00702427 -0.00702444 -0.00702427 -0.0046829 ]
 [ 0.03114082  0.0392586   0.03114082  0.02346648]] </td>
    </tr>
    <tr>
       <td> **A1** </td>
       <td> [[0.50055046 0.5021379  0.50055046 0.50089612]
 [0.50923332 0.50704555 0.50923332 0.50542658]
 [0.49824394 0.4982439  0.49824394 0.49882928]
 [0.50778458 0.50981339 0.50778458 0.50586635]] </td>
    </tr>
    <tr>
       <td> **Z2** </td>
       <td> [[-0.00426091 -0.00428969 -0.00426091 -0.00427697]
 [-0.01998143 -0.01994713 -0.01998143 -0.01990707]
 [-0.01516511 -0.01521672 -0.01516511 -0.0151588 ]] </td>
    </tr>
    <tr>
       <td> **A2** </td>
       <td> [[0.49893477 0.49892758 0.49893477 0.49893076]
 [0.49500481 0.49501338 0.49500481 0.4950234 ]
 [0.49620879 0.49619589 0.49620879 0.49621037]]</td>
</table>

Backward method test:

In [0]:
grads = nn_test.backward_propagation(x_test, y_test, cache)
print ("dW1: "+ str(grads["dW1"]))
print ("db1: "+ str(grads["db1"]))
print ("dW2: "+ str(grads["dW2"]))
print ("db2: "+ str(grads["db2"]))

**Expected Output**: 

<table>

    <tr>
       <td> **dW1** </td>
       <td> [[ 0.02797138 -0.02273175]
 [ 0.02765858  0.02777095]
 [-0.01830467 -0.01926442]
 [ 0.03115285  0.02999027]] </td>
    </tr>
    <tr>
       <td> **db1** </td>
       <td> [[0.00155825]
 [0.00133963]
 [0.00390176]
 [0.0030709 ]]</td>
    </tr>
    <tr>
       <td> **dW2** </td>
       <td> [[-0.27659717 -0.22954308 -0.27481279 -0.2806681 ]
 [-0.22820569 -0.28276153 -0.27739741 -0.28270719]
 [-0.27823467 -0.23104133 -0.27639371 -0.28252569]]
 </td>
    </tr>
    <tr>
       <td> **db2** </td>
       <td> [[-0.50106803]
 [-0.5049884 ]
 [-0.50379404]] </td>
    </tr>
</table>

Update parameters test:

In [0]:
nn_test.update_parameters(grads)
print('W1:', nn_test.W1)
print('b1:', nn_test.b1)
print('W2:', nn_test.W2)
print('b2:', nn_test.b2)

**Expected Output**: 

<table>

    <tr>
       <td> **W1** </td>
       <td> [[ 0.00468743 -0.00115533]
 [ 0.0062003   0.01495259]
 [-0.00215849 -0.00214873]
 [ 0.0154806   0.00737444]]</td>
    </tr>
    <tr>
       <td> **b1** </td>
       <td> [[-1.55825086e-05]
 [-1.33963447e-05]
 [-3.90175629e-05]
 [-3.07090141e-05]]</td>
    </tr>
    <tr>
       <td> **W2** </td>
       <td> [[-0.00192877  0.00772103 -0.00188605 -0.00185062]
 [ 0.00470168 -0.01630519 -0.0144752  -0.0027958 ]
 [-0.00734596  0.00545289 -0.0063163  -0.01129778]] </td>
    </tr>
    <tr>
       <td> **db2** </td>
       <td> [[0.00501068]
 [0.00504988]
 [0.00503794]] </td>
    </tr>
</table>

### 3.2 - Model ####

Now we can aggregate all previous modules into a classifier:

In [0]:
# GRADED CLASS NNClassifier

class NNClassifier:
    """
    NNClassifier class
    
    Arguments:
    model -- instance of NN
    epochs: int -- Number of epochs
    """
    def __init__(self, model, epochs=1000):
        self.model = model
        self.epochs = epochs
        self._cost = [] # Write value of cost function after each epoch to build graph later
    
    def fit(self, X, Y):
        """
        Learn weights and errors from training data

        Arguments:
        X -- input data of shape (number of features, number of examples)
        Y -- labels of shape (1, number of examples)
        """
    
        ### START CODE HERE ### 
        Y = one_hot(Y,self.model.n_classes)
        for epoch in range(self.epochs):
            cache = self.model.forward_propagation(X)
            grads = self.model.backward_propagation(X, Y, cache)
            self._cost.append(compute_cost(cache['A2'], Y))
            self.model.update_parameters(grads)
        ### END CODE HERE ###
    
    def predict(self, X):
        """
        Generate array of predicted labels for the input dataset
        
        Arguments:
        X -- input data of shape (number of features, number of examples)
        
        Returns:
        predicted labels of shape (1, n_samples)
        """
        
        ### START CODE HERE ### (≈ 2 lines of code)
        cache = self.model.forward_propagation(X)
        ### END CODE HERE ###

        return np.argmax(cache['A2'], axis=0).T

Accuracy:

In [0]:
def accuracy(pred, labels):
    return (np.sum(pred == labels, axis=1) / float(labels.shape[1]))[0]

Function for visualizing an error change:

In [0]:
def plot_error(model, epochs):
    plt.plot(range(len(model._cost)), model._cost)
    plt.ylim([0, epochs])
    plt.ylabel('Error')
    plt.xlabel('Epochs')
    plt.show()

## 4 - Training 

Let's initialize our classifier:

In [0]:
NN = NeuralNetwork(784, 30, 10, 0.01)
classifier = NNClassifier(NN, 5000)

After last iteration value of cost function must be <1.8:

In [0]:
classifier.fit(train_set_x, train_set_y)

In [0]:
plot_error(classifier, 10)

## 5 - Making predictions

Accuracy on the train set must be >0.95, on the test set - >0.86

In [0]:
pred_train = classifier.predict(train_set_x)
pred_test = classifier.predict(test_set_x)

print('train set accuracy: ', accuracy(pred_train, train_set_y))
print('test set accuracy: ', accuracy(pred_test, test_set_y))

Now we can check our classifier on single example:

In [0]:
plot_digit(test_set_x, test_set_y, idx=6)

In [0]:
pred_single = classifier.predict(test_set_x.T[6].reshape(784, 1))
print("The digit is " + str(pred_single[0]))

In [0]:
plot_digit(test_set_x, test_set_y, idx=90)

In [0]:
pred_single = classifier.predict(test_set_x.T[90].reshape(784, 1))
print("The digit is " + str(pred_single[0]))

## 7 - Conclusion
As we can see, our model fits well the hypothesis function to the data.

#### What's next:
1. Try to experiment with different parameters to to make prediction better.
2. Compare the results you have obtained with the `sklearn.neural_network` models.
3. Try this model in the wild! Select your favorite dataset [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=small&filetype=all&license=all&tagids=13303) and play with it.